# 미분 자동 계산


## 상자로서의 변수

### 1.1 변수란

변수란? 프로그래밍 입문서에서 변수는 상자라고 설명한다. 상자에 값을 담아두고 그 값을 사용할 수 있다.

- 상자와 데이터는 별개다
- 상자에는 데이터가 들어간다
- 상자 속을 들여다보면 데이터를 알 수 있다

### 1.2 Variable 클래스 구현

```python
class Variable:
    def __init__(self, data):
        self.data = data
```

**init** 메서드는 Variable 클래스의 생성자이다. 생성자는 인스턴스를 초기화하는 메서드이다.


In [1]:
import numpy as np


class Variable:
    def __init__(self, data):
        self.data = data


data = np.array(1.0)
x = Variable(data)
print(x.data)

1.0


머신러닝 시스템은 기본적으로 `다차원 배열`을 사용한다.


In [2]:
x.data = np.array(2.0)
print(x.data)

2.0


## 변수를 낳는 함수

### 2.1 함수란

어떤 변수로부터 다른 변수의 대응 관계를 정한 것 함수 $f(x) = x^{2}$이 있다고 할때 $y=f(x)$라고 하면 $y$와$x$의 관계가 함수 $f$에 의해 결정된다. 즉 함수 $f$에 의해 $y$는 $x$의 제곱이다 라는 관계가 성립한다.


### 2.2 Function 클래스 구현

- Function클래스는 Variable인스턴스를 입력받아 Variable인스턴스를 출력한다.
- Variable 인스턴스의 실제 데이터는 인스턴스 변수인 data에 저장된다.


In [3]:
class Function:
    def __call__(self, input):
        x = input.data
        y = x**2
        output = Variable(y)
        return output

`__call__`메서드의 인수 input은 Variable 인스턴스라고 가정한다. 따라서 실제 데이터는 input.data에 존재한다.
데이터를 꺼낸 후 원하는 계산을 하고 결과를 Variable 인스턴스로 되돌려준다.


In [4]:
x = Variable(np.array(10))
f = Function()
y = f(x)
print(type(y))
print(y.data)

<class '__main__.Variable'>
100


In [5]:
import numpy as np
from framework.variable import Variable
from framework.function import Square

x = Variable(np.array(10))
f = Square()
y = f(x)
print(type(y))
print(y.data)

<class 'framework.variable.Variable'>
100


## 함수 연결

### 3.1 Exp 함수 구현
